In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-22 04:09:17--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.007s  

2025-02-22 04:09:17 (146 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Assuming your notebook's working directory is set such that ../llm-tokenizer is reachable:
tokenizer_dir = os.path.abspath(os.path.join(os.getcwd(), '../llm-tokenizer'))
if tokenizer_dir not in sys.path:
    sys.path.insert(0, tokenizer_dir)

import BPETokenizer  # Now you should be able to import it

In [6]:
import torch
import torch.nn.functional as F
from random import random

device = 'cuda' if torch.cuda.is_available else 'cpu'

if torch.cuda.is_available():
    current_device = torch.cuda.current_device()
    device_name = torch.cuda.get_device_name(current_device)
    device_props = torch.cuda.get_device_properties(current_device)
    memory_summary = torch.cuda.memory_summary(device=current_device, abbreviated=True)
    
    print("Current device index:", current_device)
    print("Running on GPU:", device_name)
    print("GPU properties:")
    print("  - Compute Capability:", f"{device_props.major}.{device_props.minor}")
    print("  - Total Memory:", f"{device_props.total_memory / (1024**3):.2f} GB")
    print("  - Multiprocessor Count:", device_props.multi_processor_count)
    print("  - Max Threads per Multiprocessor:", device_props.max_threads_per_multi_processor)
else:
    print("CUDA is not available, running on CPU.")

Current device index: 0
Running on GPU: NVIDIA RTX A6000
GPU properties:
  - Compute Capability: 8.6
  - Total Memory: 47.54 GB
  - Multiprocessor Count: 84
  - Max Threads per Multiprocessor: 1536


# Pretraining Data - GPT Data Loader

In [7]:
class DataLoader:
    """next token prediction DataLoader used for GPT like models"""
    def __init__(self, config):
        
        self.config = config

        with open('input.txt', 'r') as f:
            text = f.read()
        len(text)

        self.tokenizer = BPETokenizer.Tokenizer(text, encoding_vocab_size=2000, raw_tokens=False)
        self.tokenizer.load_from_file()
        encoded_dataset = self.tokenizer.encode(text, raw_tokens=False)
        print(f"max vocabulary size={max(encoded_dataset)}, compression ratio={len(encoded_dataset) / len(text)}")
        split = int(len(encoded_dataset) * 0.80)
        self.train_data =  torch.tensor(encoded_dataset[:split])
        self.val_data = torch.tensor(encoded_dataset[split+config.block_size:])
        print(f"train_data.shape={self.train_data.shape}, val_data.shape={self.val_data.shape}")
        
        self.train_data_ix = 0
        self.val_data_ix = 0
        self.batch_step = self.config.batch_size * self.config.block_size 
        
    def next_batch(self, mode="train", device=device):
        """ mode=["train", "eval"] """
        if mode == "train":
            x, y = self._next_batch_train()
        else:
            x, y = self._next_batch_eval()
        if device:
            return x.to(device), y.to(device)
        return x, y
    
    def _next_batch_train(self):
        
        data = self.train_data
        ix = int(random() * (len(data) - 2*self.batch_step))
        
        buf = data[ix:ix+self.batch_step + 1]     
        x = buf[:-1].view(self.config.batch_size, self.config.block_size)
        y = buf[1:].view(self.config.batch_size, self.config.block_size)
        
        self.train_data_ix += self.batch_step 
        if self.train_data_ix + self.batch_step + 1 > len(self.train_data):
            self.train_data_ix = 0
        
        return x, y
    
    def _next_batch_eval(self):
        
        data = self.train_data
        ix = int(random() * (len(data) - 2*self.batch_step))
        
        buf = data[ix:ix+self.batch_step + 1]     
        x = buf[:-1].view(self.config.batch_size, self.config.block_size)
        y = buf[1:].view(self.config.batch_size, self.config.block_size)
        
        self.val_data_ix += self.batch_step 
        if self.val_data_ix + self.batch_step + 1 > len(self.val_data):
            self.val_data_ix = 0
        
        return x, y

In [8]:
from dataclasses import dataclass

@dataclass
class BERTConfig:
    BERT_batch_size = 6
    batch_size = BERT_batch_size * 2
    block_size = 5
    
config = BERTConfig

In [9]:
data_loader = DataLoader(config)
x, y = data_loader.next_batch(device=None)

max vocabulary size=2213, compression ratio=0.4458684554516162
train_data.shape=torch.Size([397855]), val_data.shape=torch.Size([99459])


In [10]:
x.shape, y.shape

(torch.Size([12, 5]), torch.Size([12, 5]))

# Pretraining Data - BERT Data Loader

In [100]:
class BERTDataLoader(DataLoader):
    """data loader for BERT-like MLM + NSP loss"""
    def __init__(self, config):
        super().__init__(config)
        self.max_vocab_size = max(self.tokenizer.encoding_map.values())
        self.CLS =self.max_vocab_size + 1
        self.SEP = self.CLS + 1
        self.MASK = self.SEP + 1
        print(f"new tokens: {self.max_vocab_size}, {self.CLS}, {self.SEP}, {self.MASK}")
        
    def next_batch(self, device=device, test=False):
        """return x, y for Masked Language Model (MLM), MLM loss mask, y for Next Sentence Prediction"""
        _x, _ = super().next_batch(device=None)
        x, y_MLM, ix_MLM, y_NSP = [], [], [], []
        assert len(_x) % 2 == 0, "BERTDataLoader batch size should be % 2 == 0"
        for ix in range(int(len(_x) / 2)): 
            x0 = _x[ix].clone()
            if random() < 0.5:
                x1 = _x[ix+1].clone()
                y_NSP.append(torch.tensor(1))
            else:
                __x, _ = super().next_batch(device=None)
                x1 = __x[0].clone()
                y_NSP.append(torch.tensor(0))
            y_MLM.append(self._make_input(x0, x1))
            x0, ix0 = self._mask(x0)
            x1, ix1 = self._mask(x1)
            x.append(self._make_input(x0, x1))
            ix_MLM.append(torch.cat([
                torch.tensor([0]), 
                ix0,
                torch.tensor([0]), 
                ix1,
                torch.tensor([0])
                ]))
            
        if test:
            for i, xx in enumerate(x):
                if y_NSP[i] == 1:
                    a, b = y_MLM[i][1:1+BERTConfig.block_size], _x[i]
                    assert (a == b).all(), (a, b)
                    a, b = y_MLM[i][1+BERTConfig.block_size+1:1+2*BERTConfig.block_size+1], _x[i+1]
                    assert (a == b).all(), (a, b)
                    print(f"{i} TEST PASSED same sentence")
                    
        # turn indexes of MLM into a average loss mask
        ix_MLM = torch.stack(ix_MLM).to(device)
        ix_MLM = (ix_MLM / ix_MLM.sum()).nan_to_num(0.0)
            
        return (
            torch.stack(x).to(device), 
            torch.stack(y_MLM).to(device), 
            ix_MLM, 
            torch.stack(y_NSP).to(device)
        )
    
    def _make_input(self, x0, x1):
        return torch.cat([
            torch.tensor([self.CLS]), 
            x0,
            torch.tensor([self.SEP]), 
            x1,
            torch.tensor([self.SEP])
        ])
    
    def _mask(self, x):
        ix = []
        for i, v in enumerate(x):
            if random() < 0.15:
                ix.append(torch.tensor(1))
                r2 = random()
                if r2 < 0.80:
                    x[i] = self.MASK
                elif 0.80 <= r2 < 0.90:
                    x[i] = int(random() * (self.max_vocab_size - 100))
            else: ix.append(torch.tensor(0))
        return x, torch.tensor(ix)

In [101]:
data_loader = BERTDataLoader(config)

max vocabulary size=2213, compression ratio=0.4458684554516162
train_data.shape=torch.Size([397855]), val_data.shape=torch.Size([99459])
new tokens: 2215, 2216, 2217, 2218


In [102]:
x, y_MLM, loss_mask_MLM, y_NSP = data_loader.next_batch(device=device, test=True)

0 TEST PASSED same sentence
2 TEST PASSED same sentence
4 TEST PASSED same sentence
5 TEST PASSED same sentence


# Model

In [49]:
from dataclasses import dataclass

@dataclass
class BERTConfig:
    # model is going to be (BERT_batch_size, input_size, embedding_size or hidden_layer_size)
    BERT_batch_size = 6
    batch_size = BERT_batch_size * 2
    block_size = 5
    embedding_size = 64
    vocab_size = 2220 # new tokens: 2215, 2216, 2217, 2218 from dataloader
    input_size = 2 * block_size + 3
    hidden_layer_size = 256
    attention_heads = 8
    attention_size = embedding_size // attention_heads
    
config = BERTConfig

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [51]:
def _get_x1_x2(x, config):
    return x[:, 1:1+config.block_size], x[:, 1+config.block_size+1:-1]

class InputRepresentation(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size, config.embedding_size)
        self.segment_embeddings = nn.Embedding(2, config.embedding_size)
        self.position_embeddings = nn.Embedding(config.input_size + 1, config.embedding_size)
        self.register_buffer('segment_index', 
                             torch.tensor(sum([
                                 [0] * (config.block_size + 1), 
                                 [0], 
                                 [1] * (config.block_size + 1)], [])))
        self.register_buffer('position_index', 
                             torch.arange(config.input_size))
    
    def forward(self, x):
        token_embeddings = self.token_embeddings(x)
        segment_embeddings = self.segment_embeddings(self.segment_index)
        position_embeddings = self.position_embeddings(self.position_index)
        x = position_embeddings + token_embeddings + segment_embeddings
        return x

In [52]:
input_embeddings = InputRepresentation(config)
input_embeddings.to(device)

InputRepresentation(
  (token_embeddings): Embedding(2220, 64)
  (segment_embeddings): Embedding(2, 64)
  (position_embeddings): Embedding(14, 64)
)

In [53]:
class SelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.key = nn.Linear(config.embedding_size, config.attention_size)
        self.query = nn.Linear(config.embedding_size, config.attention_size)
        self.value = nn.Linear(config.embedding_size, config.attention_size)
        
    def forward(self, x):
        key = self.key(x)
        query = self.query(x)
        attention = F.softmax(k @ q.transpose(1, 2), dim=2)
        value = self.value(x)
        return attention @ value
    
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttention(config) for _ in range(config.attention_heads)])
        self.linear = nn.Linear(config.embedding_size, config.embedding_size)
        
    def forward(self, x):
        heads = []
        for head in self.heads:
            heads.append(head(x))
        x = torch.cat(heads, dim=2)
        x = self.linear(x)
        return x

In [54]:
attention = SelfAttention(config)
attention.to(device)

attention = MultiHeadAttention(config)
attention.to(device)


MultiHeadAttention(
  (heads): ModuleList(
    (0-7): 8 x SelfAttention(
      (key): Linear(in_features=64, out_features=8, bias=True)
      (query): Linear(in_features=64, out_features=8, bias=True)
      (value): Linear(in_features=64, out_features=8, bias=True)
    )
  )
  (linear): Linear(in_features=64, out_features=64, bias=True)
)

In [113]:
class BERT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.input_representation = InputRepresentation(config)
        self.multi_head_attention = MultiHeadAttention(config)
        
        self.MLM_output = nn.Linear(config.embedding_size, config.vocab_size)
        self.NSP_output = nn.Linear(config.embedding_size, 1)
        
    def forward(self, x, y_NSP=None, y_MLM=None, loss_mask_MLM=None):
        """return (pred_NSP, logits_MSM, loss_NSP, loss_MLM)"""
        x = self.input_representation(x)
        pred_NSP = F.sigmoid(self.NSP_output(x[:, 0, :]))
        logits_MSM = self.MLM_output(x)
        
        loss_NSP, loss_MLM = None, None
        if y_NSP is not None:
            loss_NSP = F.binary_cross_entropy(pred_NSP.view(-1), y_NSP.to(torch.float))
            
        if y_MLM is not None and loss_mask_MLM is not None:
            loss_MLM = F.cross_entropy(
                logits_MSM.reshape(config.BERT_batch_size * config.input_size, -1), 
                y_MLM.reshape(-1),
                reduction='none')
            loss_MLM = (loss_MLM * loss_mask_MLM.view(-1)).sum()
        
        
        return pred_NSP, logits_MSM, loss_NSP, loss_MLM

In [114]:
model = BERT(config)
model.to(device)

BERT(
  (input_representation): InputRepresentation(
    (token_embeddings): Embedding(2220, 64)
    (segment_embeddings): Embedding(2, 64)
    (position_embeddings): Embedding(14, 64)
  )
  (multi_head_attention): MultiHeadAttention(
    (heads): ModuleList(
      (0-7): 8 x SelfAttention(
        (key): Linear(in_features=64, out_features=8, bias=True)
        (query): Linear(in_features=64, out_features=8, bias=True)
        (value): Linear(in_features=64, out_features=8, bias=True)
      )
    )
    (linear): Linear(in_features=64, out_features=64, bias=True)
  )
  (MLM_output): Linear(in_features=64, out_features=2220, bias=True)
  (NSP_output): Linear(in_features=64, out_features=1, bias=True)
)

In [115]:
x, y_MLM, loss_mask_MLM, y_NSP = data_loader.next_batch(device=device)

In [118]:
pred_NSP, logits_MSM, loss_NSP, loss_MLM = model(x, y_NSP=y_NSP, y_MLM=y_MLM, loss_mask_MLM=loss_mask_MLM)

In [119]:
loss_MLM, loss_NSP

(tensor(8.4182, device='cuda:0', grad_fn=<SumBackward0>),
 tensor(0.6389, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>))